# MODEL TRAINING - testing the full approach - COGNITION
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [3]:
samplerate=200

## load cognition

In [10]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [11]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

2785

In [12]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [13]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## define a fixed feature selection to use

In [15]:
featureSelection=loadPickle("featureSelection26.pkl")
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## load data split
10 fold CV with 4 subjects for testing

In [16]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV.pkl")
dataSplits

,train,val,test
0,"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001
1,"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002
2,"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003
3,"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004
4,"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005
5,"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006
6,"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007
7,"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008
8,"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009


## define annotation criteria

In [17]:
np.unique(annotations.labelerId)

array(['0001'], dtype=object)

In [18]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001'],
    'criteriumName':['E1'],
    'labelerIdList':[['0001']]
})

In [19]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]


In [20]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [21]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005
5,0001,E1,[0001],"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006
6,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007
7,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008
8,0001,E1,[0001],"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009


## model fitting

In [22]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))


**********************
1 of 9
**********************
2 of 9
**********************
3 of 9
**********************
4 of 9
**********************
5 of 9
**********************
6 of 9
**********************
7 of 9
**********************
8 of 9
**********************
9 of 9


In [23]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [24]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001,16b9b403-5b57-4f9b-b9b4-dd56240acf77,0.008985
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002,07e6bbaa-2aa2-4df4-8e5f-28bc716dc817,0.005890
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003,53f1c280-5951-4042-852e-00443c71b12e,0.005867
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004,e8ce0b00-6a48-4362-bc01-501a4996914c,0.008876
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005,90ab0dfb-22de-4127-bcbc-ea6d81cf7540,0.005867
5,0001,E1,[0001],"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006,fd90c505-fd19-4b2a-bbb7-a88529c36d7c,0.009365
6,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007,5278edde-a3c5-4161-9613-f4be8bd14fc0,0.009711
7,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008,81a44421-f8a4-48c9-9dba-9aa04ab97659,0.010361
8,0001,E1,[0001],"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009,f1d48762-3da9-4fe4-920f-943298435e53,0.009431


## save results

In [25]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

In [26]:
#you need to use this in the following parts of the experiment
print(experimentId)

352338b0-f723-421f-a69e-e2afb40205ed
